In [1]:
import comtypes.client
import random
import numpy as np
import os

In [2]:
# 定义用于存储所有货位矩阵和频率矩阵的列表
all_distribution_matrices = []
all_frequencies = []

In [3]:

# 连接到运行中的 SAP2000
sap_object = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
SapModel = sap_object.SapModel

# 解锁模型
SapModel.SetModelIsLocked(False)

# 设置单位
SapModel.SetPresentUnits(9)  # 设定单位为 N/mm^2


0

In [4]:
def random_goods_distribution(num_z, num_x, num_y):
    # 创建一个 num_z 行，num_x * num_y 列的矩阵
    distribution_matrix = []
    
    for layer in range(num_z):
        # 随机生成当前层的货物分布，0 或 1
        distribution = [random.choice([0, 1]) for _ in range(num_x * num_y)]
        distribution_matrix.append(distribution)  # 将当前层的分布添加到矩阵中
        
    return distribution_matrix



In [5]:
# 循环进行随机货位分布并计算频率
num_trials = 10 # 模拟次数
surface_pressure_area = 1000000*9.8 / (2000*2000)  # 均布面力，可以根据需要调整
num_z = 3
num_x = 7
num_y = 7



In [6]:
#根据货位分布矩阵，对结构施加均布面力，然后进行模态分析
num_loc = 147  # 货位总数
for trial in range(num_trials):
    # 随机分布货位
    distribution_matrix = random_goods_distribution(num_z, num_x, num_y)
    
    print(f"Trial {trial + 1}: Goods Distribution Matrix:")
    for layer in distribution_matrix:
        print(layer)

    all_distribution_matrices.append(np.array(distribution_matrix))
    flattened_distribution = [item for sublist in distribution_matrix for item in sublist]

    # 在 SAP2000 中添加均布面力
    for loc in range(num_loc):
        if flattened_distribution[loc] == 1:
            area_name = str(loc + 1)
            
            ret=sap_object.SapModel.AreaObj.SetLoadUniform(area_name,
                                                       "DEAD",
                                                       surface_pressure_area,
                                                       10,
                                                       True,
                                                       "Global",
                                                       0)
            #ret = sap_object.SapModel.AreaObj.SetLoadSurfacePressure(area_name, "DEAD", -2, 2450, " ", False, 0)
            #if ret != 0:
                #print("Area load not assigned successfully.")
            #else:
                #print("Area load assigned successfully.")
            

    #执行模态分析
    ret=SapModel.Analyze.RunAnalysis()

    #ret = SapModel.Analyze.RunAnalysis()
    #if ret != 0:
        #print("Analysis did not run successfully.")
    #else:
        #print("Analysis run successfully.")

    NumberResults = 12
    LoadCase = []
    StepType = []
    StepNum = []
    Period = []
    Frequency = []
    CircFreq = []
    EigenValue = []

    # 调用函数获取模态数据
    result = SapModel.Results.ModalPeriod(NumberResults, 
                                          LoadCase, 
                                          StepType, 
                                          StepNum, 
                                          Period, 
                                          Frequency, 
                                          CircFreq, 
                                          EigenValue)
    print(result[6])  # 6 代表CircFreq

    # 将频率矩阵添加到列表中
    all_frequencies.append(np.array(result[6]))
     #解锁模型
    SapModel.SetModelIsLocked(False)


    # 删除分析结果

    ret=sap_object.SapModel.Analyze.DeleteResults(" ", True)

    #ret = sap_object.SapModel.Analyze.DeleteResults(" ", True)
    #if ret != 0:
        #print("Results not deleted successfully.")
    #else:
        #print("Results deleted successfully.")

    # 删除均布面力
    for load in range(num_loc):
        if flattened_distribution[load] == 1:
            load_name = str(load + 1)

            ret=sap_object.SapModel.AreaObj.DeleteLoadUniform(load_name, 
                                                          "DEAD", 
                                                          0)

            #ret= sap_object.SapModel.AreaObj.DeleteLoadUniform(load_name, "DEAD", 0)
            #if ret != 0:
                #print("Area load not deleted successfully.")
            #else:
                #print("Area load deleted successfully.")

    #解锁模型
    SapModel.SetModelIsLocked(False)
    
# 创建数据存储目录
data_dir = 'train_npz'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# 将所有未经展平的货位矩阵保存到一个 npz 文件中
data_file = os.path.join(data_dir, 'data.npz')
np.savez_compressed(data_file, data=np.array(all_distribution_matrices))

# 将所有频率矩阵保存到另一个 npz 文件中
label_file = os.path.join(data_dir, 'label.npz')
np.savez_compressed(label_file, label=np.array(all_frequencies))

print("所有货位矩阵和频率矩阵已分别保存到 data.npz 和 label.npz 文件中。")
# 关闭 SAP2000
#sap_object.ApplicationExit(False)

Trial 1: Goods Distribution Matrix:
[0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
[1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
(1.389601141134068, 1.4195448047039363, 1.5713475974409836, 1.6086469483310581, 1.8172315492847624, 1.8307925247624555, 2.1076217031645497, 2.198017850622534, 2.4855026759512433, 2.533192932980773, 2.7150419548272056, 2.8201833534180323)
Trial 2: Goods Distribution Matrix:
[0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1